In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [3]:
from langchain.chat_models import init_chat_model
from elevenlabs.client import ElevenLabs
from elevenlabs import play, save
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from IPython.display import display, Markdown
from typing import List
from enum import Enum
import os

from pydantic import BaseModel, Field
import json
from typing import List

In [4]:
#model = init_chat_model("ministral-8b-latest", model_provider="mistralai", temperature=0)
model = init_chat_model("mistral-large-latest", model_provider="mistralai", temperature=0.2)
#model = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=0.3, max_retries=3)

In [11]:
class Period(str, Enum):
    """Enum for periods of life."""
    CHILDHOOD = "childhood"
    ADOLESCENCE = "adolescence"
    YOUNG_ADULTHOOD = "young_adulthood"
    ADULTHOOD = "adulthood"
    OLD_AGE = "old_age"

class Person(BaseModel):
    id: int = Field(description="The unique identifier for the person")
    name: str = Field(description="The name of the person")
    relationship: str = Field(description="The relationship of the person to the user")
    age: int = Field(description="The age of the person")
    occupation: str = Field(description="The occupation of the person")
    hobbies: List[str] = Field(default_factory=list, description="The hobbies of the person")
    description: str = Field(description="A description of the person")

    def __str__(self):
        return json.dumps(self.dict(), indent=2)

class Memory(BaseModel):
    people: List[int] = Field(min_items=1, decription="A list of IDs of people involved in the memory")
    place: List[str] = Field(min_items=1, default_factory=list, description="A list of places involved in the memory")
    period: List[str] = Field(min_items=1, default_factory=list)
    emotion: List[str] = Field(min_items=1, default_factory=list)
    description: str = Field(description="A description of the memory")

    def __str__(self):
        return json.dumps(self.dict(), indent=2)
    
class LifeSummary(BaseModel):
    name: str = Field(description="The name of the user")
    age: int = Field(description="The age of the user")
    occupation: str = Field(description="The occupation of the user")
    acquaintances: List[Person] = Field(min_items=5, default_factory=list, description="A list of acquaintances of the user (friend, colleague,  family)")
    memories: List[Memory] = Field(default_factory=list)

    def __str__(self):
        return json.dumps(self.dict(), indent=2)

# Génération des souvenirs pour la vie

In [12]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage("Tu est un chatbot qui génère des données de synthèse sur les souvenirs sur toute la vie d'une personne agée."),
        ("user", """Invente cinq souvenirs. pour une personne dont voici la description :
            Nom : {name}
            Age : {age}
            Profession : {occupation} 
         Rédige les souvenirs à la première personne,
         en utilisant le vocabulaire et les émotions d'une personne âgée.
         Chaque souvenir doit être spécifique et détaillé, en incluant la référence (id) des personnes impliquées, les lieux, la période de la vie et les émotions ressenties."""),
    ]
)
chain = prompt_template | model.with_structured_output(LifeSummary)

In [13]:
life_summary = chain.invoke({"name": "Monique", "age": "75", "occupation": "Ancienne Assistante sociale"})

In [15]:
life_summary.model_dump()

{'name': 'Monique',
 'age': 75,
 'occupation': 'Ancienne Assistante sociale',
 'acquaintances': [{'id': 1,
   'name': 'Pierre',
   'relationship': 'Mari',
   'age': 78,
   'occupation': 'Retraité',
   'hobbies': ['Jardinage', 'Lecture'],
   'description': 'Mon mari depuis 50 ans, un homme patient et aimant.'},
  {'id': 2,
   'name': 'Sophie',
   'relationship': 'Fille',
   'age': 45,
   'occupation': 'Professeur',
   'hobbies': ['Peinture', 'Randonnée'],
   'description': 'Ma fille aînée, une femme indépendante et passionnée par son métier.'},
  {'id': 3,
   'name': 'Jean',
   'relationship': 'Fils',
   'age': 42,
   'occupation': 'Ingénieur',
   'hobbies': ['Football', 'Cuisine'],
   'description': 'Mon fils cadet, un homme dévoué à sa famille et à son travail.'},
  {'id': 4,
   'name': 'Marie',
   'relationship': "Amie d'enfance",
   'age': 75,
   'occupation': 'Retraitée',
   'hobbies': ['Tricot', 'Voyages'],
   'description': "Mon amie d'enfance, toujours présente dans les moments 

# Génération de souvenirs pour un évènement particulier

In [25]:
class Person(BaseModel):
    name: str = Field(description="The name of the person")
    relationship: str = Field(description="The relationship of the person to the user")
    age: int = Field(description="The age of the person")
    occupation: str = Field(description="The occupation of the person")
    hobbies: List[str] = Field(default_factory=list, description="The hobbies of the person")
    description: str = Field(description="A description of the person")

    def __str__(self):
        return json.dumps(self.dict(), indent=2)

class Memory(BaseModel):
    person: Person = Field(decription="The writer of the memory")
    place: List[str] = Field(default_factory=list, description="A list of places involved in the memory")
    emotion: List[str] = Field(default_factory=list)
    description: str = Field(description="A description of the memory")

    def __str__(self):
        return json.dumps(self.dict(), indent=2)
    
class EnventMemories(BaseModel):
    """Memories collection about a single event with different perspectives."""
    event: str = Field(description="The specific event being remembered")
    memories: List[Memory] = Field(default_factory=list)

    def __str__(self):
        return json.dumps(self.dict(), indent=2)

In [ ]:
prompt_template = ChatPromptTemplate(
    [
        ("system", """Tu es un générateur de souvenirs narratifs, capable de simuler différents styles d’écriture, 
                      tons émotionnels et points de vue. Tu adoptes le rôle de plusieurs proches d'une personne agée pour raconter leurs souvenirs de leur point de vue pour un même évènement."""),
        ("human", """Invente {nb_memories} souvenirs riches, spécifiques et émotionnels racontés par différents proches de la personne agée et liés à un même évènement ‘‘‘{event}‘‘‘. 
                      Chaque souvenir doit être rédigé à la première personne, dans un style adapté à la personnalité du narrateur.

                      **Contexte de la personne agée** :
                        - Nom : {name}
                        - Âge : {age}
                        - Ancienne profession : {occupation}
                        - Elle a une mémoire défficiente, et ses proches souhaitent lui faire revivre des moments heureux à travers ces souvenirs.
                     
                     Rédige les souvenirs à la première personne au sujet de l'évènement : ‘‘‘{event}‘‘‘. 
                     Chaque souvenir est rédigé par un proche distinct (par exemple amis, enfants, petits-enfants, voisins, collègues retraités, etc.).
                     Chaque souvenir doit être spécifique et détaillées et en intégrant les émotions ressenties. Utilise des anecdotes.
                     """
        )
    ]
)
chain = prompt_template | model.with_structured_output(EnventMemories)

In [30]:
event_memories = chain.invoke({"name": "Monique",
"age": "75",
"occupation": "Ancienne Assistante sociale",
"event": "Mariage de Thibault, le petit fils de Monique",
"nb_memories": 15})

In [31]:
event_memories.model_dump()

{'event': 'Mariage de Thibault, le petit fils de Monique',
 'memories': [{'person': {'name': 'Monique',
    'relationship': 'Grand-mère',
    'age': 75,
    'occupation': 'Ancienne Assistante sociale',
    'hobbies': ['Jardinage', 'Lecture', 'Tricot'],
    'description': 'Une femme douce et pleine de sagesse, avec un sourire chaleureux.'},
   'place': ['Église Saint-Pierre', 'Salle des fêtes de la ville'],
   'emotion': ['Joie', 'Émotion', 'Fierté'],
   'description': "Je me souviens de ce jour comme si c'était hier. Thibault, mon petit-fils, était si élégant dans son costume noir, avec un sourire qui illuminait son visage. Quand il a dit 'oui' à sa belle, j'ai senti une vague de bonheur m'envahir. J'ai pleuré, mais ce n'étaient que des larmes de joie. La musique jouait doucement, et je me suis rappelée de mes propres noces, il y a tant d'années. C'était un moment précieux, entourée de ma famille, tous réunis pour célébrer l'amour."},
  {'person': {'name': 'Sophie',
    'relationship':

In [33]:
with open("event_memories.json", "w", encoding="utf-8") as file:
    file.write(event_memories.model_dump_json(indent=2))

# Synthèse de l'histoire

In [5]:
with open("event_memories.json", "r", encoding="utf-8") as file:
    event_memories = json.load(file)

In [6]:
class DayMemory(BaseModel):
    title: str = Field(description="The title of the memory")
    summary: str = Field(description="A summary of the memory")

In [7]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", """Tu es un assistant bienveillant spécialisé dans la stimulation de la mémoire émotionnelle chez les personnes 
                      âgées ayant une déficience de leur mémoire. Tu aides à raviver les souvenirs marquants à travers des récits narratifs, sensoriels et profondément humains."""),
        ("user", """
         Voici les informations sur la personne agée :
            Nom : {name}
            Age : {age}
            Profession : {occupation}
         Voici différents souvenirs exprimées par des proches de la personne agée au sujet de l'évènement :.
            {memories}
         Raconte cet évènement à la personne agée comme une petite histoire intime, d’environ 100 mots, qui lui parle directement.
         Utilise un ton amical, rassurant et affectueux, comme si tu étais un proche.
         Fais vivre l’histoire par les émotions, les détails sensoriels (la lumière, les couleurs, les sons, les sourires…).
         Intègre au moins une anecdote si elle est présente dans les souvenirs fournis.
         Sois doux et encourageant : le but est de faire naître un sourire, une réminiscence, une sensation familière.
         """),
    ]
)
chain = prompt_template | model.with_structured_output(DayMemory)

In [9]:
resume = chain.invoke({"name": "Monique", "age": "75", "occupation": "Ancienne Assistante sociale", "memories":event_memories})

In [10]:
resume.title

'Mariage de Thibault, le petit fils de Monique'

In [11]:
display(Markdown(resume.summary))

Monique, ma chère amie, te souviens-tu de cette journée magique où Thibault, ton petit-fils, s'est marié ? L'église Saint-Pierre était baignée d'une lumière douce, et les fleurs embaumaient l'air. Thibault, si élégant dans son costume, avait un sourire qui illuminait la pièce. Tu étais là, radieuse, les larmes aux yeux mais des larmes de joie. Tu te rappelles quand tu as pris sa main juste avant qu'il ne s'engage ? C'était un moment si émouvant. Puis, à la salle des fêtes, la musique jouait, et tu as dansé avec lui, riant aux éclats. Toute la famille était réunie, et l'amour flottait dans l'air. Tu étais si fière, si heureuse. C'était une journée inoubliable, pleine de rires et de tendresse.

In [16]:

# Initialize ElevenLabs client with your API key
api_key = os.getenv("ELEVENLABS_API_KEY")
client = ElevenLabs(api_key=api_key)
text = resume.content
voice_id = "ViSNE020Z1wEV4uZomv5"

# Generate audio from text using the specified voice ID
audio = client.generate(
    text=text,
    voice=voice_id,
    model="eleven_multilingual_v2",
    output_format="mp3_44100_128",

)

output_path = "output.mp3"

# Save the generated audio to a file
save(audio, output_path)
print(f"Audio saved to {output_path}")

Audio saved to output.mp3
